In [1]:
import pandas as pd
import numpy as np
import math
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

/Users/choejeehyuk/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [3]:
train_df = pd.read_csv('/Users/choejeehyuk/Untitled Folder/구내식당 식사 인원 예측 AI 경진대회_data/Embedding_train_data.csv')
test_df = pd.read_csv('/Users/choejeehyuk/Untitled Folder/구내식당 식사 인원 예측 AI 경진대회_data/Embedding_train_data.csv')

In [4]:
train_df.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,...,석식41,석식42,석식43,석식44,석식45,석식46,석식47,석식48,석식49,석식50
0,2016-02-01,월,2601,50,150,238,0.0,"['모닝롤', '찐빵', '우유', '두유', '주스', '계란후라이', '호두죽'...","['쌀밥', '잡곡밥', '(쌀,현미흑미:국내산)', '오징어찌개', '쇠불고기',...","['쌀밥', '잡곡밥', '(쌀,현미흑미:국내산)', '육개장', '자반고등어구이'...",...,-0.629993,0.116628,0.380194,0.250646,0.127986,0.550941,-0.815261,-0.354771,-0.467205,-0.213537
1,2016-02-02,화,2601,50,173,319,0.0,"['모닝롤', '단호박샌드', '우유', '두유', '주스', '계란후라이', '팥...","['쌀밥', '잡곡밥', '(쌀,현미흑미:국내산)', '김치찌개', '가자미튀김',...","['콩나물밥*양념장', '(쌀,현미흑미:국내산)', '어묵국', '유산슬', '(쇠...",...,0.345553,-0.856399,0.656753,0.088266,0.068821,0.651381,-0.975017,-0.925231,-0.146614,0.405589
2,2016-02-03,수,2601,56,180,111,0.0,"['모닝롤', '베이글', '우유', '두유', '주스', '계란후라이', '표고버...","['카레덮밥', '(쌀,현미흑미:국내산)', '팽이장국', '치킨핑거', '(닭고기...","['쌀밥', '잡곡밥', '(쌀,현미흑미:국내산)', '청국장찌개', '황태양념구이...",...,-0.123789,0.363554,0.589484,0.443573,0.274533,1.168624,-0.740611,-0.684546,0.015721,0.389435
3,2016-02-04,목,2601,104,220,355,0.0,"['모닝롤', '토마토샌드', '우유', '두유', '주스', '계란후라이', '닭...","['쌀밥', '잡곡밥', '(쌀,현미흑미:국내산)', '쇠고기무국', '주꾸미볶음'...","['미니김밥*겨자장', '(쌀,현미흑미:국내산)', '우동', '멕시칸샐러드', '...",...,-0.132657,-0.264607,0.894339,0.100546,0.385215,2.008252,-1.461347,-1.574167,-0.453716,0.295548
4,2016-02-05,금,2601,278,181,34,0.0,"['모닝롤', '와플', '우유', '두유', '주스', '계란후라이', '쇠고기죽...","['쌀밥', '잡곡밥', '(쌀,현미흑미:국내산)', '떡국', '돈육씨앗강정', ...","['쌀밥', '잡곡밥', '(쌀,현미흑미:국내산)', '차돌박이찌개', '(쇠고기:...",...,-0.230097,-0.053875,0.805636,0.803612,0.086424,0.482092,-1.044789,-0.500920,0.028591,-0.010523


In [5]:
train_df = train_df.drop(['조식메뉴','중식메뉴','석식메뉴'], axis = 1)
train_df.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,조식1,...,석식41,석식42,석식43,석식44,석식45,석식46,석식47,석식48,석식49,석식50
0,2016-02-01,월,2601,50,150,238,0.0,1039.0,331.0,0.155443,...,-0.629993,0.116628,0.380194,0.250646,0.127986,0.550941,-0.815261,-0.354771,-0.467205,-0.213537
1,2016-02-02,화,2601,50,173,319,0.0,867.0,560.0,0.411054,...,0.345553,-0.856399,0.656753,0.088266,0.068821,0.651381,-0.975017,-0.925231,-0.146614,0.405589
2,2016-02-03,수,2601,56,180,111,0.0,1017.0,573.0,0.257978,...,-0.123789,0.363554,0.589484,0.443573,0.274533,1.168624,-0.740611,-0.684546,0.015721,0.389435
3,2016-02-04,목,2601,104,220,355,0.0,978.0,525.0,0.515096,...,-0.132657,-0.264607,0.894339,0.100546,0.385215,2.008252,-1.461347,-1.574167,-0.453716,0.295548
4,2016-02-05,금,2601,278,181,34,0.0,925.0,330.0,0.342847,...,-0.230097,-0.053875,0.805636,0.803612,0.086424,0.482092,-1.044789,-0.500920,0.028591,-0.010523


In [6]:
test_df = test_df.drop(['조식메뉴','중식메뉴','석식메뉴'], axis = 1)
test_df.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,조식1,...,석식41,석식42,석식43,석식44,석식45,석식46,석식47,석식48,석식49,석식50
0,2016-02-01,월,2601,50,150,238,0.0,1039.0,331.0,0.155443,...,-0.629993,0.116628,0.380194,0.250646,0.127986,0.550941,-0.815261,-0.354771,-0.467205,-0.213537
1,2016-02-02,화,2601,50,173,319,0.0,867.0,560.0,0.411054,...,0.345553,-0.856399,0.656753,0.088266,0.068821,0.651381,-0.975017,-0.925231,-0.146614,0.405589
2,2016-02-03,수,2601,56,180,111,0.0,1017.0,573.0,0.257978,...,-0.123789,0.363554,0.589484,0.443573,0.274533,1.168624,-0.740611,-0.684546,0.015721,0.389435
3,2016-02-04,목,2601,104,220,355,0.0,978.0,525.0,0.515096,...,-0.132657,-0.264607,0.894339,0.100546,0.385215,2.008252,-1.461347,-1.574167,-0.453716,0.295548
4,2016-02-05,금,2601,278,181,34,0.0,925.0,330.0,0.342847,...,-0.230097,-0.053875,0.805636,0.803612,0.086424,0.482092,-1.044789,-0.500920,0.028591,-0.010523


In [7]:
train_df = train_df.rename(columns = {'일자':'date','요일':'week'})
test_df = test_df.rename(columns = {'일자':'date','요일':'week'})

In [8]:
train_df.head(2)

,date,week,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,조식1,...,석식41,석식42,석식43,석식44,석식45,석식46,석식47,석식48,석식49,석식50
0,2016-02-01,월,2601,50,150,238,0.0,1039.0,331.0,0.155443,...,-0.629993,0.116628,0.380194,0.250646,0.127986,0.550941,-0.815261,-0.354771,-0.467205,-0.213537
1,2016-02-02,화,2601,50,173,319,0.0,867.0,560.0,0.411054,...,0.345553,-0.856399,0.656753,0.088266,0.068821,0.651381,-0.975017,-0.925231,-0.146614,0.405589


In [9]:
from pandas.core.common import temp_setattr
train_df['year'] = pd.to_datetime(train_df['date']).dt.year.astype(int)
train_df['month'] = pd.to_datetime(train_df['date']).dt.month.astype(int)
train_df['day'] = pd.to_datetime(train_df['date']).dt.day.astype(int)

test_df['year'] = pd.to_datetime(test_df['date']).dt.year.astype(int)
test_df['month'] = pd.to_datetime(test_df['date']).dt.month.astype(int)
test_df['day'] = pd.to_datetime(test_df['date']).dt.day.astype(int)

In [10]:
train_df.head(2)

,date,week,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,조식1,...,석식44,석식45,석식46,석식47,석식48,석식49,석식50,year,month,day
0,2016-02-01,월,2601,50,150,238,0.0,1039.0,331.0,0.155443,...,0.250646,0.127986,0.550941,-0.815261,-0.354771,-0.467205,-0.213537,2016,2,1
1,2016-02-02,화,2601,50,173,319,0.0,867.0,560.0,0.411054,...,0.088266,0.068821,0.651381,-0.975017,-0.925231,-0.146614,0.405589,2016,2,2


In [11]:
test_df.head(2)

,date,week,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,조식1,...,석식44,석식45,석식46,석식47,석식48,석식49,석식50,year,month,day
0,2016-02-01,월,2601,50,150,238,0.0,1039.0,331.0,0.155443,...,0.250646,0.127986,0.550941,-0.815261,-0.354771,-0.467205,-0.213537,2016,2,1
1,2016-02-02,화,2601,50,173,319,0.0,867.0,560.0,0.411054,...,0.088266,0.068821,0.651381,-0.975017,-0.925231,-0.146614,0.405589,2016,2,2


In [12]:
# 기존 날짜 피처 삭제
train_df.drop(columns=['date'], inplace=True)
test_df.drop(columns=['date'], inplace=True)

train_df.shape, test_df.shape

((1205, 161), (1205, 161))

In [13]:
train_df.replace({"week": ["월","화","수","목","금"] }, {"week": [0,1,2,3,4] }, inplace = True)
test_df.replace({"week": ["월","화","수","목","금"] }, {"week": [0,1,2,3,4] }, inplace = True)
train_df.head(3)

,week,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,조식1,조식2,...,석식44,석식45,석식46,석식47,석식48,석식49,석식50,year,month,day
0,0,2601,50,150,238,0.0,1039.0,331.0,0.155443,0.749417,...,0.250646,0.127986,0.550941,-0.815261,-0.354771,-0.467205,-0.213537,2016,2,1
1,1,2601,50,173,319,0.0,867.0,560.0,0.411054,0.699290,...,0.088266,0.068821,0.651381,-0.975017,-0.925231,-0.146614,0.405589,2016,2,2
2,2,2601,56,180,111,0.0,1017.0,573.0,0.257978,0.434443,...,0.443573,0.274533,1.168624,-0.740611,-0.684546,0.015721,0.389435,2016,2,3


In [14]:
test_df.replace({"week": ["월","화","수","목","금"] }, {"week": [0,1,2,3,4] }, inplace = True)
test_df.replace({"week": ["월","화","수","목","금"] }, {"week": [0,1,2,3,4] }, inplace = True)
test_df.head(3)

,week,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,조식1,조식2,...,석식44,석식45,석식46,석식47,석식48,석식49,석식50,year,month,day
0,0,2601,50,150,238,0.0,1039.0,331.0,0.155443,0.749417,...,0.250646,0.127986,0.550941,-0.815261,-0.354771,-0.467205,-0.213537,2016,2,1
1,1,2601,50,173,319,0.0,867.0,560.0,0.411054,0.699290,...,0.088266,0.068821,0.651381,-0.975017,-0.925231,-0.146614,0.405589,2016,2,2
2,2,2601,56,180,111,0.0,1017.0,573.0,0.257978,0.434443,...,0.443573,0.274533,1.168624,-0.740611,-0.684546,0.015721,0.389435,2016,2,3


In [15]:
train_df['week'] = train_df['week'].astype('category')

In [16]:
test_df['week'] = train_df['week'].astype('category')

In [17]:
train_df.isna().sum()

week              0
본사정원수             0
본사휴가자수            0
본사출장자수            0
본사시간외근무명령서승인건수    0
                 ..
석식49              0
석식50              0
year              0
month             0
day               0
Length: 161, dtype: int64

In [18]:
#
target_col1 = '중식계'
target_col2 = '석식계'
train_cols = [col for col in train_df.columns if col not in [target_col1 , target_col2]]

###**LGBM** 

In [19]:
from sklearn.model_selection import GridSearchCV , cross_val_score
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error

In [20]:
def get_val_score(model):
    val_pred = model.predict_proba(X_valid)
    return  mean_absolute_error(y_valid,val_pred[:,1])

In [ ]:
#'중식계'

X_train, X_valid, y_train, y_valid = train_test_split(train_df[train_cols], train_df[target_col1], test_size=0.2, shuffle=True, random_state=34)

lgb_params = {
     'max_depth' : [3,5,7] , 
     'n_estimators': [300,600,900,1200], 
     'learning_rate':[0.01,0.1], 
     'gamma': [0.5], 
     'random_state':[99] }

model = lgb.LGBMRegressor(eval_set=[(X_valid, y_valid)],eval_metric = 'mae')


grid_lgb = GridSearchCV(model, param_grid=lgb_params, cv=3)
grid_lgb.fit(X_train, y_train)
print(grid_lgb.best_params_)

#Get score
print("="*20)
#print("LGBM Validation MAE : ", get_val_score(grid_lgb))
print("Model1 Score: ",grid_lgb.score(X_valid,y_valid))
print("MAE:",mean_absolute_error(y_valid, grid_lgb.predict(X_valid)))

In [23]:
pred1 = grid_lgb.predict(test_df)

ValueError: Number of features of the model must match the input. Model n_features_ is 159 and input n_features is 161 

In [ ]:
pred1

In [ ]:
#'석식계'

X_train, X_valid, y_train, y_valid = train_test_split(train_df[train_cols], train_df[target_col2], test_size=0.2, shuffle=True, random_state=34)

lgb_params = {
     'max_depth' : [3,5,7] , 
     'n_estimators': [1000,1200,1500], 
     'learning_rate':[0.01,0.1], 
     'gamma': [0.5], 
     'random_state':[99] } 

model = lgb.LGBMRegressor(eval_set=[(X_valid, y_valid)],eval_metric = 'mae')


grid_lgb = GridSearchCV(model, param_grid=lgb_params, cv=3)
grid_lgb.fit(X_train, y_train)
print(grid_lgb.best_params_)

#Get score
print("="*20)
print("Model2 Score: ",grid_lgb.score(X_valid,y_valid))
print("MAE:",mean_absolute_error(y_valid, grid_lgb.predict(X_valid)))

In [ ]:
pred2 = grid_lgb.predict(test_df)

In [ ]:
pred2

###**LinearReg**

In [ ]:
from sklearn.linear_model import LinearRegression

In [93]:
# '중식계'

X_train, X_valid, y_train, y_valid = train_test_split(train_df[train_cols], train_df[target_col1], test_size=0.2, shuffle=True, random_state=34)

lin_reg = LinearRegression()

lin_reg.fit(X_train,y_train)
y_valid_predict = lin_reg.predict(X_valid)

print("Model3 Score: ",lin_reg.score(X_valid,y_valid))
print("MAE:",mean_absolute_error(y_valid, lin_reg.predict(X_valid)))

Model3 Score:  0.727622124834058
MAE: 84.37756109587967


In [94]:
pred3 = lin_reg.predict(test_df)

In [95]:
pred3

array([1009.27593444, 1075.31746829,  790.82969396, 1154.4895111 ,
        896.14090441,  963.36612572,  984.03318554,  841.41766831,
       1213.78342321,  906.05244642,  798.69393761, 1111.73127169,
       1187.9911585 , 1023.71766862, 1027.65967718,  700.37065613,
       1017.71352328, 1067.16605844,  902.62620993,  959.92501724,
        493.64399106, 1222.53720112, 1000.05515476,  988.71985384,
        775.15451199, 1062.26390624, 1210.25764818,  787.01161236,
       1079.35822309,  796.15760865, 1118.91737262, 1125.96319864,
       1049.3174058 ,  970.04818647,  668.58621764, 1089.15505291,
       1183.54976034,  877.72429135,  813.24620861,  631.77280653,
        994.62467849, 1035.83299109,  912.31029461,  994.01859163,
        652.08579549, 1008.23319651, 1170.95455107, 1072.58993399,
        941.37537666,  596.05653737])

In [96]:
# '석식계'

X_train, X_valid, y_train, y_valid = train_test_split(train_df[train_cols], train_df[target_col2], test_size=0.2, shuffle=True, random_state=34)

lin_reg = LinearRegression()

lin_reg.fit(X_train,y_train)
y_valid_predict = lin_reg.predict(X_valid)


print("Model4 Score: ",lin_reg.score(X_valid,y_valid))
print("MAE:",mean_absolute_error(y_valid, lin_reg.predict(X_valid)))

Model4 Score:  0.6755819638002729
MAE: 58.500190789470096


In [97]:
pred4 = lin_reg.predict(test_df)

In [98]:
pred4

array([286.10596878, 142.58707396, 307.05663669, 564.73818188,
       359.05819939,  24.5498509 , 449.07920762, 358.48571106,
       481.25375748, 410.73045506, 313.87374151, 567.43083737,
       538.80479786, 442.50574464, 448.2332223 , 242.08622712,
       633.03774413, 519.24102826, 436.91698441, 545.7993373 ,
       319.18249338, 544.75384646, 359.48540877, 587.88784845,
       375.56263594, 674.86368903, 569.70252353, 315.64708572,
       191.78502776, 364.9110915 , 698.42669583, 502.78231917,
       419.05661993, 556.21093389, 340.58095835, 637.63737584,
       505.28119621, 310.4475846 , 309.56632795, 336.04994102,
       643.82332259, 452.52101629, 309.28726141, 442.93943134,
       308.16555174, 608.81297065, 495.05527552, 403.31572289,
       394.46033574, 291.70403288])